In [1]:
!pip install -q torch peft==0.4.0 bitsandbytes transformers trl accelerate einops tqdm scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer
# import evaluate
import torch
import numpy as np
from dataclasses import dataclass, field
from huggingface_hub import interpreter_login

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset, load_from_disk

from transformers import (
    AutoTokenizer,
    AutoConfig,
	  AutoModelForCausalLM,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
	  BitsAndBytesConfig,
	  HfArgumentParser,
    TrainingArguments)

In [ ]:
dataset = load_dataset("b-mc2/sql-create-context", split='train')

def format_rows(row):
    question = f"{row['question']} context: {row['context']}"
    answer = {row['answer']}
    return f"[INST] {question} [/INST] {answer}"

README.md:   0%|          | 0.00/4.43k [00:00<?, ?B/s]

sql_create_context_v4.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

In [ ]:
df = pd.DataFrame(dataset)

df['text'] = df.apply(format_rows, axis = 1)

data_df = df['text']

In [ ]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)  # Show full column content
data_df.head(5)

,Text
0,[INST] How many heads of the departments are older than 56 ? context: CREATE TABLE head (age INTEGER) [/INST] {'SELECT COUNT(*) FROM head WHERE age > 56'}
1,"[INST] List the name, born state and age of the heads of departments ordered by age. context: CREATE TABLE head (name VARCHAR, born_state VARCHAR, age VARCHAR) [/INST] {'SELECT name, born_state, age FROM head ORDER BY age'}"
2,"[INST] List the creation year, name and budget of each department. context: CREATE TABLE department (creation VARCHAR, name VARCHAR, budget_in_billions VARCHAR) [/INST] {'SELECT creation, name, budget_in_billions FROM department'}"
3,"[INST] What are the maximum and minimum budget of the departments? context: CREATE TABLE department (budget_in_billions INTEGER) [/INST] {'SELECT MAX(budget_in_billions), MIN(budget_in_billions) FROM department'}"
4,"[INST] What is the average number of employees of the departments whose rank is between 10 and 15? context: CREATE TABLE department (num_employees INTEGER, ranking INTEGER) [/INST] {'SELECT AVG(num_employees) FROM department WHERE ranking BETWEEN 10 AND 15'}"


In [ ]:
data_df.to_csv("sql_training_data.csv", index = False)
training_dataset = load_dataset("csv", data_files = "sql_training_data.csv", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
base_model = "microsoft/phi-4"
new_model = "master-sql-syntax-phi-4"
tokenizer = AutoTokenizer.from_pretrained(base_model, add_prefix_space=True, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/917k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.25M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [ ]:
bits_and_bites_config = BitsAndBytesConfig(load_in_4bit = True, bnd_4bit_quant_type = "nf4", bnd_4bit_compute_dtype = torch.float16, bnd_4bit_use_double_quant = False)

Unused kwargs: ['bnd_4bit_quant_type', 'bnd_4bit_compute_dtype', 'bnd_4bit_use_double_quant']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


In [ ]:
!pip install -U bitsandbytes

In [ ]:
model = AutoModelForCausalLM.from_pretrained(base_model,
                                             quantization_config = bits_and_bites_config,
                                             trust_remote_code = True,
                                             low_cpu_mem_usage = True)


config.json:   0%|          | 0.00/820 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/4.77G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.77G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [ ]:
model.config.use_cache = False
model.config.pretraining_tp = 1
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

In [ ]:
training_arguments= TrainingArguments( output_dir= "microsoft_phi_4_ft",
  num_train_epochs=0.01,
  per_device_train_batch_size=2,
	gradient_accumulation_steps=32,
	evaluation_strategy="no",
	eval_steps = 100,
	logging_steps = 15,
	optim = "paged_adamw_8bit",
  learning_rate=2e-4,
	lr_scheduler_type = "cosine",
	save_steps=1500,
	warmup_ratio = 0.05,
	weight_decay=0.01,
  eval_strategy = "no")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# print(training_dataset.column_names)
# training_dataset = training_dataset.rename_column("Text", "text")
print(training_dataset.column_names)

['text']


In [ ]:
peft_lora_config  = LoraConfig(r = 32, lora_alpha = 64, lora_dropout = 0.05, task_type = "CAUSAL_LM", target_modules = ["q_proj", "k_proj", "v_proj", "dense"])

trainer = SFTTrainer(model = model, train_dataset = training_dataset, peft_config = peft_lora_config,  tokenizer=tokenizer, args =training_arguments )


<ipython-input-17-6857aa64435b>:3: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(model = model, train_dataset = training_dataset, peft_config = peft_lora_config,  tokenizer=tokenizer, args =training_arguments )
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/78577 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss


Step,Training Loss


TrainOutput(global_step=13, training_loss=2.225385225736178, metrics={'train_runtime': 2589.7, 'train_samples_per_second': 0.303, 'train_steps_per_second': 0.005, 'total_flos': 5125373291335680.0, 'train_loss': 2.225385225736178, 'epoch': 0.010588205350097992})

In [ ]:
!huggingface-cli login
model_id = "Niranjanrao/microsoft_phi_4_ft"
model.push_to_hub(model_id) # save model
tokenizer.push_to_hub(model_id)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

model-00003-of-00003.safetensors:   0%|          | 0.00/2.06G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.80G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Niranjanrao/microsoft_phi_4_ft/commit/e3769c24ab54ed8267b34a62cbcb0a66f68721ed', commit_message='Upload tokenizer', commit_description='', oid='e3769c24ab54ed8267b34a62cbcb0a66f68721ed', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Niranjanrao/microsoft_phi_4_ft', endpoint='https://huggingface.co', repo_type='model', repo_id='Niranjanrao/microsoft_phi_4_ft'), pr_revision=None, pr_num=None)

#### How to connect your Laptop/local Ollama from Google Collab

In [10]:
import requests
import json

# Replace with your ngrok URL
OLLAMA_SERVER_URL = "https://05d2-2601-283-4700-5d00-6d97-4045-aa6c-d255.ngrok-free.app/"

# Example: Run a query
prompt = "Generate an SQL query to list all albums with more than 5 tracks."
response = requests.post(
    f"{OLLAMA_SERVER_URL}/api/generate",
    json={"model": "llama3.2", "prompt": prompt}
)
raw_content = response.text
response_lines = raw_content.splitlines()
combined_response = ""
for line in response_lines:
        try:
            data = json.loads(line)  # Parse JSON line
            if "response" in data:
                combined_response += data["response"]  # Append the response field
        except json.JSONDecodeError:
            print(f"Error decoding JSON line: {line}")
print(f"Ollama response: {combined_response}")

Ollama response: Here's a simple SQL query that lists all albums with more than 5 tracks:

```sql
SELECT album_name, num_tracks
FROM albums
WHERE num_tracks > 5;
```

In this query:

- `album_name` is the column in your table where you have stored the names of albums.
- `num_tracks` is the column that stores the number of tracks in each album.
- The `WHERE` clause filters records from the `albums` table based on whether the number of tracks (`num_tracks`) is greater than 5.

Please note that this query assumes that your database schema already has these columns.


In [35]:
modelfile_content = """
name: microsoft_phi_4_ft
model_path: ./microsoft_phi_4_ft
description: Fine-tuned Microsoft Phi 4 model for SQL generation.
"""

with open("./microsoft_phi_4_ft/modelfile", "w") as f:
    f.write(modelfile_content)

In [36]:
!ls -l /content/microsoft_phi_4_ft/

total 9505536
-rw-r--r-- 1 root root       1339 Jan 11 21:42 config.json
-rw-r--r-- 1 root root        143 Jan 11 21:42 generation_config.json
-rw-r--r-- 1 root root     916646 Jan 11 21:45 merges.txt
-rw-r--r-- 1 root root 4990541426 Jan 11 21:44 model-00001-of-00002.safetensors
-rw-r--r-- 1 root root 4733320326 Jan 11 21:45 model-00002-of-00002.safetensors
-rw-r--r-- 1 root root        125 Jan 11 21:49 modelfile
-rw-r--r-- 1 root root        125 Jan 11 21:48 Modelfile
-rw-r--r-- 1 root root      67483 Jan 11 21:45 model.safetensors.index.json
-rw-r--r-- 1 root root        579 Jan 11 21:45 special_tokens_map.json
-rw-r--r-- 1 root root      17826 Jan 11 21:45 tokenizer_config.json
-rw-r--r-- 1 root root    7153083 Jan 11 21:45 tokenizer.json
-rw-r--r-- 1 root root    1612637 Jan 11 21:45 vocab.json


In [45]:
import requests

# Replace with your ngrok URL
NGROK_URL = "https://05d2-2601-283-4700-5d00-6d97-4045-aa6c-d255.ngrok-free.app/"
MODEL_NAME = "microsoft_phi_4_ft"
MODEL_PATH = "./microsoft_phi_4_ft"  # Path to the model in Colab

# Create the model
response = requests.post(
    f"{NGROK_URL}/api/create",
    json={"model": MODEL_NAME, "modelfile": "FROM llama3.2"}
)

if response.status_code == 200:
    print(f"Model {MODEL_NAME} created successfully.")
else:
    print(f"Failed to create model: {response.status_code}, {response.text}")

Model microsoft_phi_4_ft created successfully.


In [46]:
import requests

# Replace with your ngrok URL
NGROK_URL = "https://05d2-2601-283-4700-5d00-6d97-4045-aa6c-d255.ngrok-free.app/"
MODEL_NAME = "microsoft_phi_4_ft"
MODEL_PATH = "./microsoft_phi_4_ft"  # Path to the model in Colab

# Create the model
response = requests.post(
    f"{NGROK_URL}/api/push",
    json={"model": MODEL_NAME}
)

if response.status_code == 200:
    print(f"Model {MODEL_NAME} posted successfully.")
else:
    print(f"Failed to posted model: {response.status_code}, {response.text}")

Model microsoft_phi_4_ft created successfully.
